In [2]:
from pymongo import MongoClient
import pandas as pd
import numpy as np

client = MongoClient("localhost")
db = client.fit
# Issue the serverStatus command and print the results
server_status_result = db.command("serverStatus")
    

In [3]:
cursor = db.activity.find({'message_global_mesg_num': 20, 'message_type':'FitDataMessage'})

In [4]:
df = pd.DataFrame(list(cursor))

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1789261 entries, 0 to 1789260
Data columns (total 47 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   _id                         object 
 1   message_type                object 
 2   message_chunk               object 
 3   message_frame_type          int64  
 4   message_global_mesg_num     int64  
 5   message_isDeveloperData     bool   
 6   message_local_mesg_num      int64  
 7   message_name                object 
 8   message_timeOffset          object 
 9   timestamp                   object 
 10  position_lat                object 
 11  position_long               object 
 12  distance                    float64
 13  accumulated_power           float64
 14  enhanced_altitude           float64
 15  altitude                    float64
 16  enhanced_speed              float64
 17  speed                       float64
 18  power                       float64
 19  unknown_87           

In [6]:
df.replace(to_replace='None',value=np.nan, inplace = True)

In [7]:
df['position_lat'] = df['position_lat'].apply(pd.to_numeric)

In [8]:
df['position_long'] = df['position_long'].apply(pd.to_numeric)

In [9]:
df = df[df['position_lat'].notna()]

In [10]:
df = df[df['speed'].notna()]

In [11]:
df = df[df['cadence'].notna()]

In [12]:
df = df[df['heart_rate'].notna()]

In [13]:
df = df.drop(['eE','unknown_87','message_timeOffset','message_chunk','message_isDeveloperData'], 1)

In [14]:
df['left_right_balance'].replace(to_replace='right', value=np.nan, inplace=True)

In [15]:
pd.to_numeric(df['left_right_balance'])

0            NaN
1            NaN
2            NaN
3            NaN
4            NaN
           ...  
1789256    154.0
1789257    154.0
1789258      NaN
1789259      NaN
1789260      NaN
Name: left_right_balance, Length: 1616885, dtype: float64

In [16]:
df = df.drop(['activity_type','distance_1','distance_2'], 1)

In [17]:
df['timestamp'] = df['timestamp'].apply(pd.to_datetime)

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1616885 entries, 0 to 1789260
Data columns (total 39 columns):
 #   Column                      Non-Null Count    Dtype              
---  ------                      --------------    -----              
 0   _id                         1616885 non-null  object             
 1   message_type                1616885 non-null  object             
 2   message_frame_type          1616885 non-null  int64              
 3   message_global_mesg_num     1616885 non-null  int64              
 4   message_local_mesg_num      1616885 non-null  int64              
 5   message_name                1616885 non-null  object             
 6   timestamp                   1616885 non-null  datetime64[ns, UTC]
 7   position_lat                1616885 non-null  float64            
 8   position_long               1616885 non-null  float64            
 9   distance                    1616885 non-null  float64            
 10  accumulated_power           13

In [19]:
df.to_csv('garmin_data_clean_level_1_v4.csv')

In [20]:
cursor = db.activity.find({'message_global_mesg_num': 18, 'message_type':'FitDataMessage'})

In [21]:
df_summary = pd.DataFrame(list(cursor))

In [22]:
df_summary.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 557 entries, 0 to 556
Columns: 137 entries, _id to total_caloriesAV
dtypes: bool(1), float64(118), int64(4), object(14)
memory usage: 592.5+ KB


In [23]:
df_summary.shape

(557, 137)

In [24]:
print(df_summary.dtypes[:50])

_id                              object
message_type                     object
message_chunk                    object
message_frame_type                int64
message_global_mesg_num           int64
message_isDeveloperData            bool
message_local_mesg_num            int64
message_name                     object
message_timeOffset               object
timestamp                        object
start_time                       object
start_position_lat              float64
start_position_long             float64
total_elapsed_time              float64
total_timer_time                float64
total_distance                  float64
total_strokes                   float64
nec_lat                         float64
nec_long                        float64
swc_lat                         float64
swc_long                        float64
unknown_38                      float64
unknown_39                      float64
avg_stroke_count                float64
total_work                      float64


In [25]:
df_summary['timestamp'] = df_summary['timestamp'].apply(pd.to_datetime)

In [26]:
df_summary['start_time'] = df_summary['start_time'].apply(pd.to_datetime)

In [27]:
print(df_summary.dtypes[100:])

pool_length_unit                   float64
avg_temperature                    float64
max_temperature                    float64
unknown_81                         float64
avg_fractional_cadence             float64
max_fractional_cadence             float64
total_fractional_cycles            float64
avg_left_torque_effectiveness      float64
avg_right_torque_effectiveness     float64
avg_left_pedal_smoothness          float64
avg_right_pedal_smoothness         float64
avg_combined_pedal_smoothness      float64
unknown_109                        float64
avg_left_pco                       float64
avg_right_pco                      float64
avg_cadence_position_1             float64
avg_cadence_position_2             float64
max_cadence_position_1             float64
max_cadence_position_2             float64
total_anaerobic_training_effect    float64
unknown_138_1                      float64
unknown_138_2                      float64
unknown_150                        float64
unknown_184

In [28]:
df_summary.to_csv('garmin_activity_summary_clean_v1.csv')